In [1]:
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 729, done.
remote: Counting objects: 100% (729/729), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 7248 (delta 466), reused 539 (delta 355), pack-reused 6519
Receiving objects: 100% (7248/7248), 8.63 MiB | 19.33 MiB/s, done.
Resolving deltas: 100% (5191/5191), done.


In [2]:
%cd deep-person-reid

/content/deep-person-reid


In [3]:
!python setup.py develop

/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
Compiling torchreid/metrics/rank_cylib/rank_cy.pyx because it changed.
[1/1] Cythonizing torchreid/metrics/rank_cylib/rank_cy.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/deep-person-reid/torchreid/metrics/rank_cylib/rank_cy.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running develop
running egg_info
creating torchreid.egg-info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
writing manifest file 'torchreid.egg-info/SOURCES.txt'

In [0]:
import torchreid

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [5]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 8738 KB/s, 17 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

In [6]:
teacher_model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

print(f"Number of parameters in resnet-50 - {count_parameters(teacher_model)}")

teacher_model = teacher_model.cuda()

optimizer = torchreid.optim.build_optimizer(
    teacher_model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 89.7MB/s]


In [21]:
print(f"Number of parameters in resnet-50 - {count_parameters(teacher_model)}")

Number of parameters in resnet-50 - 25046831


In [0]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    teacher_model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [0]:
engine.run(
    save_dir='log/resnet50',
    max_epoch=30,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training
Epoch: [1/30][10/404]	Time 0.453 (0.522)	Data 0.000 (0.040)	Loss 7.3906 (6.8005)	Acc 0.00 (0.00)	Lr 0.000300	eta 1:45:24
Epoch: [1/30][20/404]	Time 0.458 (0.490)	Data 0.000 (0.020)	Loss 6.6644 (6.9188)	Acc 0.00 (0.16)	Lr 0.000300	eta 1:38:44
Epoch: [1/30][30/404]	Time 0.461 (0.479)	Data 0.000 (0.014)	Loss 6.5841 (6.8413)	Acc 0.00 (0.10)	Lr 0.000300	eta 1:36:30
Epoch: [1/30][40/404]	Time 0.454 (0.473)	Data 0.000 (0.010)	Loss 6.6529 (6.7909)	Acc 0.00 (0.08)	Lr 0.000300	eta 1:35:16
Epoch: [1/30][50/404]	Time 0.464 (0.470)	Data 0.001 (0.008)	Loss 6.6231 (6.7550)	Acc 0.00 (0.06)	Lr 0.000300	eta 1:34:35
Epoch: [1/30][60/404]	Time 0.455 (0.468)	Data 0.000 (0.007)	Loss 6.6514 (6.7275)	Acc 0.00 (0.10)	Lr 0.000300	eta 1:34:03
Epoch: [1/30][70/404]	Time 0.458 (0.467)	Data 0.000 (0.006)	Loss 6.6187 (6.7044)	Acc 0.00 (0.18)	Lr 0.000300	eta 1:33:42
Epoch: [1/30][80/404]	Time 0.454 (0.466)	Data 0.000 (0.005)	Loss 6.3813 (6.6921)	Acc 3.12 (0.31)	Lr 0.000300	eta 1:33:26
Epoch: [1/30][

In [0]:
torch.save(teacher_model.state_dict(), "teacher_model.pth")

In [0]:
!mv ../teacher_model.pth teacher_model.pth

In [8]:
import torch
teacher_model.load_state_dict(torch.load("teacher_model.pth"))

<All keys matched successfully>

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class KnowledgeDistillationLoss(nn.Module):

    def __init__(self, temperature, alpha):
        super(KnowledgeDistillationLoss, self).__init__()
        self.get_cross_entropy_loss = nn.CrossEntropyLoss()
        self.get_KL_divergence_loss = nn.KLDivLoss(reduction='batchmean')
        self.temperature = temperature
        self.alpha = alpha
        # self.teacher_model = teacher_model
    
    def forward(self, teacher_outputs, student_logits, labels):
        # teacher_outputs = self.teacher_model(input_images)
        cross_entropy_loss = self.get_cross_entropy_loss(student_logits, labels)
        kl_divergence_loss = self.get_KL_divergence_loss(F.log_softmax(student_logits/self.temperature, dim=1), F.softmax(teacher_outputs/self.temperature, dim=1))
        total_loss = cross_entropy_loss*self.alpha + kl_divergence_loss*(1 - self.alpha)*self.temperature*self.temperature
        # import pdb;pdb.set_trace()
        return total_loss

In [0]:
import time
import numpy as np
import os.path as osp
import datetime
import torch
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
from torchreid.engine import Engine
from torchreid import metrics
from torchreid.utils import (
    AverageMeter, re_ranking, save_checkpoint, visualize_ranked_results
)
from torchreid.utils import (
    AverageMeter, open_all_layers, open_specified_layers
)
# from torchreid.losses import DeepSupervision
from torchreid.losses import CrossEntropyLoss

class ImageKnowledgeDistillationEngine(Engine):
    def __init__(
        self,
        datamanager,
        model,
        optimizer,
        teacher_model,
        alpha,
        temperature,
        scheduler=None,
        use_gpu=True,
        label_smooth=True
    ):
        super(ImageKnowledgeDistillationEngine, self
              ).__init__(datamanager, model, optimizer, scheduler, use_gpu)

        self.temperature = temperature
        self.alpha = alpha
        self.teacher_model = teacher_model

        self.criterion = KnowledgeDistillationLoss(self.temperature, self.alpha)

    def train(
        self,
        epoch,
        max_epoch,
        writer,
        print_freq=10,
        fixbase_epoch=0,
        open_layers=None
    ):
        losses = AverageMeter()
        accs = AverageMeter()
        batch_time = AverageMeter()
        data_time = AverageMeter()

        self.model.train()
        if (epoch + 1) <= fixbase_epoch and open_layers is not None:
            print(
                '* Only train {} (epoch: {}/{})'.format(
                    open_layers, epoch + 1, fixbase_epoch
                )
            )
            open_specified_layers(self.model, open_layers)
        else:
            open_all_layers(self.model)

        num_batches = len(self.train_loader)
        end = time.time()
        for batch_idx, data in enumerate(self.train_loader):
            data_time.update(time.time() - end)

            imgs, pids = self._parse_data_for_train(data)
            if self.use_gpu:
                imgs = imgs.cuda()
                pids = pids.cuda()

            self.optimizer.zero_grad()
            outputs = self.model(imgs)
            with torch.no_grad():
                teacher_outputs = self.teacher_model(imgs)
            loss = self.criterion(teacher_outputs, outputs, pids)
            loss.backward()
            self.optimizer.step()

            batch_time.update(time.time() - end)

            losses.update(loss.item(), pids.size(0))
            accs.update(metrics.accuracy(outputs, pids)[0].item())

            if (batch_idx+1) % print_freq == 0:
                # estimate remaining time
                eta_seconds = batch_time.avg * (
                    num_batches - (batch_idx+1) + (max_epoch -
                                                   (epoch+1)) * num_batches
                )
                eta_str = str(datetime.timedelta(seconds=int(eta_seconds)))
                print(
                    'Epoch: [{0}/{1}][{2}/{3}]\t'
                    'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                    'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                    'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                    'Acc {acc.val:.2f} ({acc.avg:.2f})\t'
                    'Lr {lr:.6f}\t'
                    'eta {eta}'.format(
                        epoch + 1,
                        max_epoch,
                        batch_idx + 1,
                        num_batches,
                        batch_time=batch_time,
                        data_time=data_time,
                        loss=losses,
                        acc=accs,
                        lr=self.optimizer.param_groups[0]['lr'],
                        eta=eta_str
                    )
                )

            if writer is not None:
                n_iter = epoch*num_batches + batch_idx
                writer.add_scalar('Train/Time', batch_time.avg, n_iter)
                writer.add_scalar('Train/Data', data_time.avg, n_iter)
                writer.add_scalar('Train/Loss', losses.avg, n_iter)
                writer.add_scalar('Train/Acc', accs.avg, n_iter)
                writer.add_scalar(
                    'Train/Lr', self.optimizer.param_groups[0]['lr'], n_iter
                )

            end = time.time()

        if self.scheduler is not None:
            self.scheduler.step()

In [17]:
squeezenet_baseline_model = torchreid.models.build_model(
    name='squeezenet1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

squeezenet_baseline_model = squeezenet_baseline_model.cuda()

print(f"Number of parameters in squeezenet - {count_parameters(squeezenet_baseline_model)}")

optimizer = torchreid.optim.build_optimizer(
    squeezenet_baseline_model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    squeezenet_baseline_model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir='log/squeezenet1_0',
    max_epoch=30,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

torch.save(squeezenet_baseline_model.state_dict(), "squeezenet_baseline_model.pth")

Number of parameters in squeezenet - 1120687
=> Start training
Epoch: [1/30][10/404]	Time 0.076 (0.107)	Data 0.000 (0.043)	Loss 6.6361 (6.6936)	Acc 0.00 (0.00)	Lr 0.000300	eta 0:21:31
Epoch: [1/30][20/404]	Time 0.073 (0.081)	Data 0.000 (0.022)	Loss 6.6150 (6.6558)	Acc 0.00 (0.16)	Lr 0.000300	eta 0:16:25
Epoch: [1/30][30/404]	Time 0.052 (0.074)	Data 0.000 (0.015)	Loss 6.5903 (6.6400)	Acc 0.00 (0.21)	Lr 0.000300	eta 0:14:56
Epoch: [1/30][40/404]	Time 0.049 (0.071)	Data 0.000 (0.011)	Loss 6.6069 (6.6352)	Acc 0.00 (0.16)	Lr 0.000300	eta 0:14:19
Epoch: [1/30][50/404]	Time 0.064 (0.068)	Data 0.000 (0.009)	Loss 6.6047 (6.6283)	Acc 3.12 (0.19)	Lr 0.000300	eta 0:13:36
Epoch: [1/30][60/404]	Time 0.047 (0.065)	Data 0.000 (0.008)	Loss 6.5931 (6.6253)	Acc 0.00 (0.16)	Lr 0.000300	eta 0:12:59
Epoch: [1/30][70/404]	Time 0.066 (0.065)	Data 0.000 (0.007)	Loss 6.6096 (6.6219)	Acc 0.00 (0.13)	Lr 0.000300	eta 0:13:03
Epoch: [1/30][80/404]	Time 0.046 (0.064)	Data 0.000 (0.006)	Loss 6.5868 (6.6140)	Acc 0.00 

In [18]:
squeezenet_distilled = torchreid.models.build_model(
    name='squeezenet1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

squeezenet_distilled = squeezenet_distilled.cuda()

print(f"Number of parameters in squeezenet - {count_parameters(squeezenet_distilled)}")

optimizer = torchreid.optim.build_optimizer(
    squeezenet_distilled,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = ImageKnowledgeDistillationEngine(
    datamanager,
    squeezenet_distilled,
    optimizer=optimizer,
    scheduler=scheduler,
    teacher_model=teacher_model,
    alpha=0.01,
    temperature=20,
    label_smooth=True
)

engine.run(
    save_dir='log/knowledge_distillation',
    max_epoch=30,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

torch.save(squeezenet_distilled.state_dict(), "squeezenet_distilled.pth")

Number of parameters in squeezenet - 1120687
=> Start training
Epoch: [1/30][10/404]	Time 0.042 (0.107)	Data 0.000 (0.036)	Loss 0.7093 (0.7528)	Acc 0.00 (0.31)	Lr 0.000300	eta 0:21:36
Epoch: [1/30][20/404]	Time 0.041 (0.076)	Data 0.000 (0.018)	Loss 0.7301 (0.7348)	Acc 0.00 (0.31)	Lr 0.000300	eta 0:15:14
Epoch: [1/30][30/404]	Time 0.058 (0.065)	Data 0.000 (0.012)	Loss 0.6956 (0.7259)	Acc 0.00 (0.21)	Lr 0.000300	eta 0:13:04
Epoch: [1/30][40/404]	Time 0.045 (0.059)	Data 0.000 (0.009)	Loss 0.6907 (0.7187)	Acc 0.00 (0.23)	Lr 0.000300	eta 0:11:54
Epoch: [1/30][50/404]	Time 0.046 (0.056)	Data 0.000 (0.008)	Loss 0.6911 (0.7146)	Acc 0.00 (0.19)	Lr 0.000300	eta 0:11:15
Epoch: [1/30][60/404]	Time 0.037 (0.054)	Data 0.000 (0.006)	Loss 0.6929 (0.7111)	Acc 0.00 (0.21)	Lr 0.000300	eta 0:10:53
Epoch: [1/30][70/404]	Time 0.036 (0.053)	Data 0.000 (0.005)	Loss 0.6769 (0.7085)	Acc 0.00 (0.22)	Lr 0.000300	eta 0:10:35
Epoch: [1/30][80/404]	Time 0.049 (0.051)	Data 0.000 (0.005)	Loss 0.6961 (0.7067)	Acc 0.00 

In [27]:
0.0123*(15913+3368)/32

7.4111343750000005

In [19]:
mobilenet_baseline = torchreid.models.build_model(
    name='mobilenetv2_x1_4',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

mobilenet_baseline = mobilenet_baseline.cuda()

print(f"Number of parameters in mobilenet - {count_parameters(mobilenet_baseline)}")

optimizer = torchreid.optim.build_optimizer(
    mobilenet_baseline,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    mobilenet_baseline,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir='log/mobilenet_no_kd',
    max_epoch=30,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

torch.save(mobilenet_baseline.state_dict(), "mobilenet_baseline.pth")

Number of parameters in mobilenet - 5638147
=> Start training


/content/deep-person-reid/torchreid/models/mobilenetv2.py:272: UserWarning: The imagenet pretrained weights need to be manually downloaded from https://mega.nz/#!RGhgEIwS!xN2s2ZdyqI6vQ3EwgmRXLEW3khr9tpXg96G9SUJugGk
  .format(model_urls['mobilenetv2_x1_4'])


Epoch: [1/30][10/404]	Time 0.087 (0.138)	Data 0.000 (0.042)	Loss 6.9539 (6.7539)	Acc 0.00 (0.94)	Lr 0.000300	eta 0:27:55
Epoch: [1/30][20/404]	Time 0.085 (0.113)	Data 0.000 (0.021)	Loss 7.0991 (6.9011)	Acc 0.00 (0.62)	Lr 0.000300	eta 0:22:47
Epoch: [1/30][30/404]	Time 0.090 (0.105)	Data 0.000 (0.014)	Loss 6.9666 (6.9955)	Acc 0.00 (0.52)	Lr 0.000300	eta 0:21:04
Epoch: [1/30][40/404]	Time 0.085 (0.100)	Data 0.000 (0.011)	Loss 6.8727 (7.0149)	Acc 0.00 (0.47)	Lr 0.000300	eta 0:20:06
Epoch: [1/30][50/404]	Time 0.089 (0.097)	Data 0.000 (0.009)	Loss 6.7857 (6.9702)	Acc 0.00 (0.38)	Lr 0.000300	eta 0:19:31
Epoch: [1/30][60/404]	Time 0.091 (0.096)	Data 0.000 (0.007)	Loss 6.9814 (6.9230)	Acc 0.00 (0.52)	Lr 0.000300	eta 0:19:13
Epoch: [1/30][70/404]	Time 0.088 (0.095)	Data 0.000 (0.006)	Loss 6.5178 (6.8816)	Acc 3.12 (0.54)	Lr 0.000300	eta 0:18:59
Epoch: [1/30][80/404]	Time 0.087 (0.094)	Data 0.000 (0.005)	Loss 6.6423 (6.8411)	Acc 3.12 (0.70)	Lr 0.000300	eta 0:18:48
Epoch: [1/30][90/404]	Time 0.089

In [20]:
mobile_net_kd = torchreid.models.build_model(
    name='mobilenetv2_x1_4',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

mobile_net_kd = mobile_net_kd.cuda()

print(f"Number of parameters in mobilenet - {count_parameters(mobile_net_kd)}")

optimizer = torchreid.optim.build_optimizer(
    mobile_net_kd,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = ImageKnowledgeDistillationEngine(
    datamanager,
    mobile_net_kd,
    optimizer=optimizer,
    scheduler=scheduler,
    teacher_model=teacher_model,
    alpha=0.01,
    temperature=20,
    label_smooth=True
)

engine.run(
    save_dir='log/knowledge_distillation',
    max_epoch=30,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

torch.save(mobile_net_kd.state_dict(), "mobile_net_kd.pth")

Number of parameters in mobilenet - 5638147
=> Start training


/content/deep-person-reid/torchreid/models/mobilenetv2.py:272: UserWarning: The imagenet pretrained weights need to be manually downloaded from https://mega.nz/#!RGhgEIwS!xN2s2ZdyqI6vQ3EwgmRXLEW3khr9tpXg96G9SUJugGk
  .format(model_urls['mobilenetv2_x1_4'])


Epoch: [1/30][10/404]	Time 0.115 (0.166)	Data 0.000 (0.040)	Loss 0.6949 (0.7076)	Acc 0.00 (0.00)	Lr 0.000300	eta 0:33:25
Epoch: [1/30][20/404]	Time 0.116 (0.142)	Data 0.000 (0.020)	Loss 0.6971 (0.7033)	Acc 0.00 (0.47)	Lr 0.000300	eta 0:28:37
Epoch: [1/30][30/404]	Time 0.115 (0.133)	Data 0.000 (0.014)	Loss 0.6836 (0.7026)	Acc 3.12 (0.52)	Lr 0.000300	eta 0:26:51
Epoch: [1/30][40/404]	Time 0.117 (0.129)	Data 0.000 (0.010)	Loss 0.6977 (0.7005)	Acc 0.00 (0.62)	Lr 0.000300	eta 0:25:58
Epoch: [1/30][50/404]	Time 0.115 (0.127)	Data 0.000 (0.008)	Loss 0.7041 (0.6984)	Acc 0.00 (0.56)	Lr 0.000300	eta 0:25:26
Epoch: [1/30][60/404]	Time 0.115 (0.125)	Data 0.000 (0.007)	Loss 0.7158 (0.6975)	Acc 0.00 (0.47)	Lr 0.000300	eta 0:25:04
Epoch: [1/30][70/404]	Time 0.115 (0.123)	Data 0.000 (0.006)	Loss 0.6793 (0.6951)	Acc 0.00 (0.54)	Lr 0.000300	eta 0:24:47
Epoch: [1/30][80/404]	Time 0.115 (0.122)	Data 0.000 (0.005)	Loss 0.6809 (0.6941)	Acc 3.12 (0.70)	Lr 0.000300	eta 0:24:33
Epoch: [1/30][90/404]	Time 0.116

In [24]:
%%time
engine.test(1)

##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3368-by-1792 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-1792 matrix
Speed: 0.0293 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 60.7%
CMC curve
Rank-1  : 79.4%
Rank-5  : 91.1%
Rank-10 : 94.3%
Rank-20 : 96.4%
CPU times: user 12.7 s, sys: 2.94 s, total: 15.7 s
Wall time: 34.4 s


0.793943

In [28]:
0.0293*(15913+3368)/32

17.654165625

In [29]:
0.0316*(15913+3368)/32

19.039987500000002

In [30]:
0.0122*(15913+3368)/32

7.3508812500000005

In [0]:
sqeezenet_baseline_engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    squeezenet_baseline_model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [26]:
%%time
sqeezenet_baseline_engine.test(1)

##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Speed: 0.0137 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 29.6%
CMC curve
Rank-1  : 54.6%
Rank-5  : 73.7%
Rank-10 : 81.5%
Rank-20 : 87.1%
CPU times: user 10.8 s, sys: 3.11 s, total: 13.9 s
Wall time: 32.7 s


0.5463183

In [0]:
teacher_baseline_engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    teacher_model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [32]:
teacher_baseline_engine.test(1)

##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3368-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-2048 matrix
Speed: 0.0294 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 67.3%
CMC curve
Rank-1  : 83.8%
Rank-5  : 93.9%
Rank-10 : 96.0%
Rank-20 : 97.3%


0.8375891

In [0]:
!python train_baseline.py --model_type squeezenet1_0 --num_epochs 3

In [16]:
!python train_knowledge_distillation_model.py --model_type squeezenet1_0 --teacher_type resnet50 --num_epochs 10

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal